<a href="https://colab.research.google.com/github/VelasquezE/ML4Sci_2025-II/blob/main/Module2/clasficadorTexto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Clasificador de texto**

# 0. Importar las librerías necesarias

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import FunctionTransformer
from sklearn.model_selection import cross_val_score

import pickle
from ipywidgets import FileUpload, VBox, Button, Output, Text
import io

In [ ]:
# Personalización gráficas

sns.set_style("darkgrid")

custom_rc = {
    "xtick.bottom": True,
    "ytick.left": True,
    "xtick.direction": "out",
    "ytick.direction": "out",
    "axes.edgecolor": "black",
    "grid.color": "#bbbbbb",
    "grid.linestyle": "dashed",
    "axes.grid": True,
    "axes.spines.top": True,
    "axes.spines.right": False,
    "axes.spines.left": False,
    "axes.spines.bottom": True
}
sns.set_context("notebook")
sns.set_style("darkgrid", rc = custom_rc)

# 1. Cargar los datos de entrenamiento

Se descargan los archivos csv de un repositorio de GitHub.
Luego, se carga el contenido en un DataFrame de pandas para su posterior análisis.

In [ ]:
def get_data(name):
    """
    Descarga un archivo csv dado el nombre del archivo.
    Carga los datos en un DataFrame.

    Returns:
        pandas.DataFrame: Datos extraídos.
    """
    base_url = "https://raw.githubusercontent.com/VelasquezE/ML4Sci_2025-II/refs/heads/main/Module2/"
    url = base_url + name
    output_file = name
    !wget -O {output_file} {url}

    data = pd.read_csv(output_file)
    return data

In [ ]:
file_name = "exc_03_train.csv"

data = get_data(file_name)

Obsérvese la información básica de los datos,

In [ ]:
data.head()

In [ ]:
data.info()

In [ ]:
data["Topic"].value_counts()

# 2. Pre-procesamiento de los datos

Se empieza limpiando los datos. Se eliminan los caracteres especiales (números, puntuación, símbolos), se dejan las palabras en minúscula, la lematización convierte una palabra a su forma base (cars -> car), y quita las "stopwords" es decir "the, a, an, in, on, with...".

In [ ]:
nltk.download("stopwords")
nltk.download("wordnet")
nltk.download("omw-1.4")

In [ ]:
stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()

def clean_text(text: str):
  """
  Receives a text and deletes special caracters,
  stop words, turns to lowercase, and applies
  lemmatization.

  Parameters:
    text (str)

  Returns:
    Clean text.
  """
  text = re.sub('[^a-zA-Z]', ' ', text)
  text = text.lower()
  words = text.split()
  words = [lemmatizer.lemmatize(w) for w in words if w not in stop_words]

  return  " ".join(words)


def clean_text_batch(texts):
  return [clean_text(t) for t in texts]

# 3. Crear pipeline

Se crea un pipeline que realizará los siguientes pasos:
1.   Aplicará la función creada para limpieza del texto
2.   Vectoriza el texto en palabras
3.   Aplica el modelo de clasificación



In [ ]:
def create_pipeline(classifier):
  pipeline = Pipeline([
      ("cleaner", FunctionTransformer(clean_text_batch)),
      ("tfidf", TfidfVectorizer()),
      ("clf", classifier)
  ])

  return pipeline

# 4. División de los datos

Se dividen los datos en un dataset de entrenamiento (80%) y otro de prueba (20%).

In [ ]:
seed = 19
test_size = 0.2

X_train, X_test, y_train, y_test = train_test_split(data["Comment"], data["Topic"], test_size=test_size, random_state=seed)

# 5. Creación de los modelos

Se aplica `MultinomialNB()` y `ComplementNB()`.

In [ ]:
pipe_nb = create_pipeline(MultinomialNB())
pipe_nbc = create_pipeline(ComplementNB())

pipe_nb.fit(X_train, y_train)
predict_nb = pipe_nb.predict(X_test)
pipe_nbc.fit(X_train, y_train)
predict_nbc = pipe_nbc.predict(X_test)

# 6. Evaluación de los modelos

Se calcula la precisión,

In [ ]:
accuracy_nb = accuracy_score(y_test, predict_nb)
accuracy_nbc = accuracy_score(y_test, predict_nbc)

print(f'Accuracy MultinomialNB(): {(accuracy_nb * 100):.2f}%')
print(f'Accuracy ComplementNB(): {(accuracy_nbc * 100):.2f}%')


Se grafica la matriz de confusión,

In [ ]:
conf_matrix_nb = confusion_matrix(y_test, predict_nb)
conf_matrix_nbc = confusion_matrix(y_test, predict_nbc)

class_labels = np.unique(y_train)

fig, (ax1, ax2) = plt.subplots(1, 2)

sns.heatmap(conf_matrix_nb,
            ax = ax1,
            annot=True,
            fmt='d',
            cmap='crest',
            cbar = False,
            xticklabels=class_labels,
            yticklabels=class_labels)
sns.heatmap(conf_matrix_nbc,
            ax = ax2,
            annot=True,
            fmt='d',
            cmap='crest',
            cbar = False,
            xticklabels=class_labels,
            yticklabels=class_labels)
ax1.set_title('MultinomialNB')
ax2.set_title('ComplementNB')
fig.suptitle('Matriz de confusión')
fig.supxlabel('Etiqueta predicha')
fig.supylabel('Etiqueta verdadera')
fig.savefig("confusion_matrix.jpg", dpi = 400)
plt.show()

Y, se aplica validación cruzada,

In [ ]:
models = [
    ("MultinomialNB", MultinomialNB()),
    ("ComplementNB", ComplementNB()),
]

for name, clf in models:
    pipeline = create_pipeline(clf)
    scores = cross_val_score(pipeline, data["Comment"], data["Topic"], cv=5, scoring='accuracy')
    print(f"{name} → Mean accuracy: {scores.mean():.4f}")

# 7. Selección de modelo final

El método `ComplementNB()` fue el que realizó una mejor clasificación. Así, este se presentará como modelo final. Además, ya que se hicieron las pruebas, se entrena con todo el conjunto de datos.

In [ ]:
model = create_pipeline(ComplementNB())
model.fit(data["Comment"], data["Topic"])

Se guarda el modelo entrenado con `pickle`.

In [ ]:
filename = 'text_classifier_model.sav'
pickle.dump(model, open(filename, 'wb'))

Se crea una función que permita al usuario ingresar nuevos mensajes:

In [ ]:
def predict_category(text:str, model=model):
  predicted_label = model.predict([text])

  print(f"The input text belongs to the '{predicted_label[0]}' category.")

In [ ]:
new_message = "I love spacecrafts"
predict_category(new_message, model)

In [ ]:
new_message = "Hormones are crazy"
predict_category(new_message, model)

También, para que sea cómodo para el usuario, se crea una pequeña interfaz que solicite un nuevo conjunto de datos y calcule la matriz de confusión.

In [ ]:
uploader = FileUpload(accept='.csv', multiple=False)

print("Por favor suba el archivo y escriba los nombres de las columnas para identificar\n"+
      "cuál corresponde a los comentarios y cuál a las etiquetas:")

text_column = Text(description="Columna texto: ",
                        style = {'description_width': 'initial'})
label_column = Text(description="Columna etiquetas: ",
                         style = {'description_width': 'initial'})

button = Button(description="Evaluar modelo")
out = Output()

def evaluate_model(model):
    out.clear_output()
    with out:
        if not uploader.value:
            print("Por favor, suba un archivo CSV primero.")
            return

        uploaded = list(uploader.value.values())[0]
        data = pd.read_csv(io.BytesIO(uploaded['content']))
        text_column_name = text_column.value.strip()
        label_column_name = label_column.value.strip()

        if text_column_name not in data.columns:
            print(f"La columna '{text_column_name}' no existe en el archivo.")
            return

        if label_column_name not in data.columns:
            print(f"La columna '{label_column_name}' no existe en el archivo.")
            return

        X_test = data[text_column_name]
        y_true = data[label_column_name]

        y_pred = model.predict(X_test)

        accuracy = accuracy_score(y_true, y_pred)
        print(f'Accuracy: {(accuracy * 100):.2f}%')

        cm = confusion_matrix(y_true, y_pred)
        class_labels = np.unique(y_true)

        sns.heatmap(cm,
            annot=True,
            fmt='d',
            cmap='crest',
            cbar = False,
            xticklabels=class_labels,
            yticklabels=class_labels)
        plt.title('Confusion Matrix')
        plt.xlabel('Predicted Label')
        plt.ylabel('True Label')
        plt.show()


button.on_click(lambda event: evaluate_model(model))

display(VBox([uploader, text_column, label_column, button, out]))